<a href="https://colab.research.google.com/github/selgebali/Colabs/blob/main/Plotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DataCite Metrics September 2024

In [1]:
!pip install plotly
!pip install -U kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 6.4 MB/s eta 0:00:00


##Cumulative sum of registered and findable DOIs over the years.

In [2]:
import plotly.graph_objects as go
import pandas as pd

# Load the relevant data from the Google Sheet (replace with your actual sheet ID)
sheet_id = '1PQe813L2M-aR_5i-1gEbHtP4M4iclgdedhdrj9DuhNg'
sheet_name = 'DOIs_OT'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

# Load the spreadsheet
spreadsheet = pd.read_csv(url)

# Clean the data
spreadsheet = spreadsheet.dropna()  # Drop rows with missing values
spreadsheet.iloc[:, 0] = spreadsheet.iloc[:, 0].astype(str)  # Ensure "Years" are string
cumulative_sum = spreadsheet.iloc[:, 1].replace({',': ''}, regex=True).astype(int)  # Clean cumulative sum data

# Plotly interactive bar graph for Cumulative sum of DOIs with values displayed
fig = go.Figure()

# Create the bar graph with text on bars
fig.add_trace(go.Bar(
    x=spreadsheet.iloc[:, 0],  # Years
    y=cumulative_sum,  # Cumulative sum
    name='Cumulative Sum',
    marker=dict(color='#243b54'),
    text=cumulative_sum,  # Display the cumulative sum values
    textposition='outside'  # place text (inside or outside)

))

# Update layout for better readability
fig.update_layout(
    title='Cumulative Sum of Registered DOIs Over the Years',
    xaxis_title='Year',
    yaxis_title='Cumulative Sum of DOIs',
    font=dict(size=14),
    hovermode='closest',
    width=1500,  # Specify the width of the figure
    height=800,   # Specify the height of the figure
    plot_bgcolor='white',  # Set plot background to white
    paper_bgcolor='white',  # Set entire figure background to white
)
# Update x-axis and y-axis to set axis lines and ticks to light grey
fig.update_xaxes(
    showline=True,  # Show x-axis line
    linecolor='lightgrey',  # Set x-axis line color to light grey
    showgrid=False  # Optionally remove grid lines for x-axis
)

fig.update_yaxes(
    showline=True,  # Show y-axis line
    linecolor='lightgrey',  # Set y-axis line color to light grey
    tickformat=',',  # Add commas to large numbers
    showgrid=False  # Optionally remove grid lines for y-axis
)
# Show the interactive plotly figure
fig.show()

## Number of DOIs register per year

In [3]:
import plotly.graph_objects as go
import pandas as pd

# Load the relevant data from the Google Sheet (replace with your actual sheet ID)
sheet_id = '1PQe813L2M-aR_5i-1gEbHtP4M4iclgdedhdrj9DuhNg'
sheet_name = 'DOIs_yearly'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

# Load the spreadsheet
spreadsheet = pd.read_csv(url)

# Clean the data
def clean_data(df):
    """Cleans the input DataFrame by handling missing values, ensuring integers, and formatting numbers."""
    df = df.dropna()  # Drop rows with missing data
    df.iloc[:, 0] = df.iloc[:, 0].astype(int)  # Ensure 'Years' is integer
    df.iloc[:, 1] = df.iloc[:, 1].replace({',': ''}, regex=True).astype(int)  # Ensure cumulative sum is integer
    return df

spreadsheet = clean_data(spreadsheet)

# Extract the relevant columns
years = spreadsheet.iloc[:, 0]  # Column 0: "Years"
registered = spreadsheet.iloc[:, 1]  # Column 1: "Number of DOIs Registered"

# Plotly interactive bar graph for the number of DOIs registered each year
fig = go.Figure()

# Create the bar graph with values on top
fig.add_trace(go.Bar(
    x=years,  # Years
    y=registered,  # Number of DOIs registered
    name='DOIs Registered',
    marker=dict(color='#243b54'),
    text=registered,  # Display the number of DOIs registered
    textposition='outside'  # Automatically place text
))

# Update layout for better readability
fig.update_layout(
    title='Number of DOIs Registered Each Year',
    xaxis_title='Year',
    yaxis_title='Number of DOIs Registered',
    font=dict(size=18),
    hovermode='closest',
    width=1500,  # Specify the width of the figure
    height=800,   # Specify the height of the figure
    plot_bgcolor='white',  # Set plot background to white
    paper_bgcolor='white',  # Set entire figure background to white
)
# Update x-axis and y-axis to set axis lines and ticks to light grey
fig.update_xaxes(
    showline=True,  # Show x-axis line
    linecolor='lightgrey',  # Set x-axis line color to light grey
    showgrid=False  # Optionally remove grid lines for x-axis
)

fig.update_yaxes(
    showline=True,  # Show y-axis line
    linecolor='lightgrey',  # Set y-axis line color to light grey
    tickformat=',',  # Add commas to large numbers
    showgrid=False  # Optionally remove grid lines for y-axis
)
# Show the interactive plotly figure
fig.show()

## Metadata Completeness

In [4]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Function to load the data from Google Sheets
def load_data(sheet_id, sheet_name):
    """Loads the data from the Google Sheets."""
    url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
    df = pd.read_csv(url)

    # Set the first column (Properties) as the index
    df = df.set_index(df.columns[0])

    # Slice the dataframe to include only columns A to T (i.e., first 19 columns)
    df = df.iloc[:, :19]

    # Convert the data to floats and multiply by 100 to get percentages
    df = df.apply(lambda col: col.map(lambda x: float(x) * 100 if pd.notnull(x) else 0))

    return df

# Function to plot the stacked bar chart
def plot_stacked_bar_chart(df):
    """Plots a stacked bar chart of Present vs Missing for each property."""
    properties = df.columns.tolist()  # List of properties
    present = df.loc['Present'].values[0]  # Use only the first row for Present values
    missing = df.loc['Missing'].values[0]  # Use only the first row for Missing values

    # Debugging: print the data to verify
    #print("Present (after correction):", present)
    #print("Missing (after correction):", missing)

    # Create the stacked bar chart with Plotly
    fig = go.Figure()

    # Add Present bars
    fig.add_trace(go.Bar(
        x=properties,
        y=present,
        name='Present',
        marker=dict(color='#243b54'),

        text=[f'<b>{p:.2f}%</b>' for p in present],  # Add percentage labels for Present
        textposition='auto'  # Automatically place text inside bars
    ))

    # Add Missing bars
    fig.add_trace(go.Bar(
        x=properties,
        y=missing,
        name='Missing',
        marker=dict(color='#00B1E2'),
        #base=present,  # Stack missing on top of present
        #text=[f'<b>{m:.2f}%</b>' for m in missing],  # Add percentage labels for Missing
        #textposition='outside'  # Automatically place text inside bars
    ))

    # Update layout for better readability
    fig.update_layout(
        barmode='stack',
        title='Metadata Completeness by Property',
        #xaxis_title='Property',
        yaxis_title='Percentage',
        font=dict(size=16),
        hovermode='x unified',
        xaxis_tickangle=-45,  # Rotate x-axis labels for readability
        width=1500,  # Specify the width of the figure
        height=800,   # Specify the height of the figure
        plot_bgcolor='white',  # Set plot background to white
        paper_bgcolor='white',  # Set entire figure background to white
    )


    # Show the interactive plotly figure
    fig.show()

# Main script
if __name__ == "__main__":
    # Google Sheets ID and sheet name
    sheet_id = '1PQe813L2M-aR_5i-1gEbHtP4M4iclgdedhdrj9DuhNg'
    sheet_name = 'Properties'

    # Load the data from Google Sheets
    df = load_data(sheet_id, sheet_name)

    # Plot the stacked bar chart
    plot_stacked_bar_chart(df)

In [5]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Function to load the data from Google Sheets
def load_data(sheet_id, sheet_name):
    """Loads the data from the Google Sheets."""
    url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
    df = pd.read_csv(url)

    # Set the first column (Properties) as the index
    df = df.set_index(df.columns[0])

    # Slice the dataframe to include only columns A to T (i.e., first 5 columns)
    df = df.iloc[:, :19]

    # Convert the data to floats and multiply by 100 to get percentages
    df = df.apply(lambda col: col.map(lambda x: float(x) * 100 if pd.notnull(x) else 0))

    return df

# Function to plot the stacked bar chart
def plot_stacked_bar_chart(df):
    """Plots a horizontal stacked bar chart of Present vs Missing for each property."""
    # Extract the 'Present' and 'Missing' rows
    present = df.loc['Present'].values[0]  # Use only the first row for Present values
    missing = df.loc['Missing'].values[0]  # Use only the first row for Missing values
    properties = df.columns.tolist()  # List of properties

    # Create a DataFrame for sorting
    df_sorted = pd.DataFrame({
        'Properties': properties,
        'Present': present,
        'Missing': missing
    })

    # Sort the DataFrame by 'Present' in descending order
    df_sorted = df_sorted.sort_values(by='Present', ascending=True)

    # Extract the sorted properties, present, and missing values
    sorted_properties = df_sorted['Properties'].values
    sorted_present = df_sorted['Present'].values
    sorted_missing = df_sorted['Missing'].values

    # Create the stacked horizontal bar chart with Plotly
    fig = go.Figure()

    # Add Present bars (horizontal orientation)
    fig.add_trace(go.Bar(
        y=sorted_properties,  # Set y-axis to properties (categories) for horizontal bars
        x=sorted_present,
        name='Present',
        orientation='h',  # Horizontal bars
        marker=dict(color='#243b54'),
        text=[f'<b>{p:.2f}%</b>' for p in sorted_present],  # Add percentage labels for Present
        textposition='inside',  # Place text inside the bars for better readability
        insidetextanchor='end',  # Ensures text is well-placed for horizontal bars
    ))

    # Add Missing bars (horizontal orientation)
    fig.add_trace(go.Bar(
        y=sorted_properties,  # Set y-axis to properties (categories) for horizontal bars
        x=sorted_missing,
        name='Missing',
        orientation='h',  # Horizontal bars
        marker=dict(color='#00B1E2'),
        base=sorted_present,  # Stack missing on top of present
        text=[f'<b>{m:.2f}%</b>' for m in sorted_missing],  # Add percentage labels for Missing
        textposition='none',  # Hide text for Missing (optional)
    ))

    # Update layout for better readability and tighter spacing
    fig.update_layout(
        barmode='stack',
        title='Metadata Completeness by Property (Horizontal, Sorted by Present)',
        xaxis_title='Percentage',
        #yaxis_title='Property',
        font=dict(size=16),  # Reduced font size slightly for cleaner look
        hovermode='y unified',
        width=900,  # Slightly reduced width
        height=500,  # Reduced height to compress the chart vertically
        plot_bgcolor='white',  # Set plot background to white
        paper_bgcolor='white',  # Set entire figure background to white
        bargap=0.1,  # Almost zero gap between bars of different categories
        margin=dict(l=80, r=20, t=40, b=40),  # Adjust margins to reduce empty space further
        showlegend=True,  # Legend for clarity
    )

    # Show the interactive plotly figure
    fig.show()

# Main script
if __name__ == "__main__":
    # Google Sheets ID and sheet name
    sheet_id = '1PQe813L2M-aR_5i-1gEbHtP4M4iclgdedhdrj9DuhNg'
    sheet_name = 'Properties'

    # Load the data from Google Sheets
    df = load_data(sheet_id, sheet_name)

    # Plot the stacked horizontal bar chart
    plot_stacked_bar_chart(df)

In [6]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Function to load the data from Google Sheets
def load_data(sheet_id, sheet_name):
    """Loads the data from the Google Sheets."""
    url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
    df = pd.read_csv(url)

    # Set the first column (Properties) as the index
    df = df.set_index(df.columns[0])

    # Slice the dataframe to include only columns A to T (i.e., first 5 columns)
    df = df.iloc[:, :19]

    # Convert the data to floats and multiply by 100 to get percentages
    df = df.apply(lambda col: col.map(lambda x: float(x) * 100 if pd.notnull(x) else 0))

    return df

# Function to assign properties to categories
def assign_categories():
    """Defines the categories for the properties."""
    mandatory_properties = ['Creators', 'Titles', 'Publisher', 'Publication Year', 'Resource Type']
    recommended_properties = ['Subjects', 'Contributors', 'Dates', 'RelatedIdentifier', 'Description', 'GeoLocation']
    optional_properties = ['Language', 'AlternateIdentifier', 'Size', 'Format', 'Version', 'Rights', 'Funding References', 'RelatedItem']

    return mandatory_properties, recommended_properties, optional_properties

# Function to plot the stacked bar chart
def plot_stacked_bar_chart(df):
    """Plots a horizontal stacked bar chart of Present vs Missing for each property."""

    # Extract the 'Present' and 'Missing' rows
    present = df.loc['Present'].values[0]  # Use only the first row for Present values
    missing = df.loc['Missing'].values[0]  # Use only the first row for Missing values
    properties = df.columns.tolist()  # List of properties

    # Create a DataFrame for sorting
    df_sorted = pd.DataFrame({
        'Properties': properties,
        'Present': present,
        'Missing': missing
    })

    # Sort the DataFrame by 'Present' in descending order
    df_sorted = df_sorted.sort_values(by='Present', ascending=True)

    # Extract the sorted properties, present, and missing values
    sorted_properties = df_sorted['Properties'].values
    sorted_present = df_sorted['Present'].values
    sorted_missing = df_sorted['Missing'].values

    # Create the stacked horizontal bar chart with Plotly
    fig = go.Figure()

    # Add Present bars (horizontal orientation)
    fig.add_trace(go.Bar(
        y=sorted_properties,  # Set y-axis to properties (categories) for horizontal bars
        x=sorted_present,
        name='Present',
        orientation='h',  # Horizontal bars
        marker=dict(color='#243b54'),
        text=[f'<b>{p:.2f}%</b>' for p in sorted_present],  # Add percentage labels for Present
        textposition='inside',  # Place text inside the bars for better readability
        insidetextanchor='end',  # Ensures text is well-placed for horizontal bars
    ))

    # Add Missing bars (horizontal orientation)
    fig.add_trace(go.Bar(
        y=sorted_properties,  # Set y-axis to properties (categories) for horizontal bars
        x=sorted_missing,
        name='Missing',
        orientation='h',  # Horizontal bars
        marker=dict(color='#00B1E2'),
        base=sorted_present,  # Stack missing on top of present
        text=[f'<b>{m:.2f}%</b>' for m in sorted_missing],  # Add percentage labels for Missing
        textposition='none',  # Hide text for Missing (optional)
    ))

    # Assign categories to properties
    mandatory_properties, recommended_properties, optional_properties = assign_categories()

    # Create a mapping of colors based on the categories
    property_colors = []
    for prop in sorted_properties:
        if prop in mandatory_properties:
            property_colors.append('#243b54')  # Mandatory category color
        elif prop in recommended_properties:
            property_colors.append('#BC2B66')  # Recommended category color
        elif prop in optional_properties:
            property_colors.append('#0D60D4')  # Optional category color

    # Update layout with colored y-axis labels
    fig.update_layout(
        barmode='stack',
        title='Metadata Completeness by Property (Horizontal, Sorted by Present)',
        xaxis_title='Percentage',
        font=dict(size=16),  # Font size for the chart
        hovermode='y unified',
        width=900,  # Chart width
        height=500,  # Chart height
        plot_bgcolor='white',  # Set plot background to white
        paper_bgcolor='white',  # Set entire figure background to white
        bargap=0.1,  # Small gap between bars
        margin=dict(l=120, r=20, t=40, b=40),  # Adjust margins
        showlegend=True,  # Show legend
        yaxis=dict(
            tickmode='array',  # Custom tick values and text for y-axis
            tickvals=sorted_properties,  # Y-axis labels (properties)
            ticktext=[f"<span style='color:{color};'>{prop}</span>" for prop, color in zip(sorted_properties, property_colors)],  # Apply colors to properties
        )
    )

    # Show the interactive plotly figure
    fig.show()

# Main script
if __name__ == "__main__":
    # Google Sheets ID and sheet name
    sheet_id = '1PQe813L2M-aR_5i-1gEbHtP4M4iclgdedhdrj9DuhNg'
    sheet_name = 'Properties'

    # Load the data from Google Sheets
    df = load_data(sheet_id, sheet_name)

    # Plot the stacked horizontal bar chart
    plot_stacked_bar_chart(df)

## Mandatory properties only

In [7]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Function to load the data from Google Sheets
def load_data(sheet_id, sheet_name):
    """Loads the data from the Google Sheets."""
    url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
    df = pd.read_csv(url)

    # Set the first column (Properties) as the index
    df = df.set_index(df.columns[0])

    # Slice the dataframe to include only columns A to T (i.e., first 5 columns)
    df = df.iloc[:, :5]

    # Convert the data to floats and multiply by 100 to get percentages
    df = df.apply(lambda col: col.map(lambda x: float(x) * 100 if pd.notnull(x) else 0))

    return df

# Function to plot the stacked bar chart
def plot_stacked_bar_chart(df):
    """Plots a horizontal stacked bar chart of Present vs Missing for each property."""
    # Extract the 'Present' and 'Missing' rows
    present = df.loc['Present'].values[0]  # Use only the first row for Present values
    missing = df.loc['Missing'].values[0]  # Use only the first row for Missing values
    properties = df.columns.tolist()  # List of properties

    # Create a DataFrame for sorting
    df_sorted = pd.DataFrame({
        'Properties': properties,
        'Present': present,
        'Missing': missing
    })

    # Sort the DataFrame by 'Present' in descending order
    df_sorted = df_sorted.sort_values(by='Present', ascending=True)

    # Extract the sorted properties, present, and missing values
    sorted_properties = df_sorted['Properties'].values
    sorted_present = df_sorted['Present'].values
    sorted_missing = df_sorted['Missing'].values

    # Create the stacked horizontal bar chart with Plotly
    fig = go.Figure()

    # Add Present bars (horizontal orientation)
    fig.add_trace(go.Bar(
        y=sorted_properties,  # Set y-axis to properties (categories) for horizontal bars
        x=sorted_present,
        name='Present',
        orientation='h',  # Horizontal bars
        marker=dict(color='#243b54'),
        text=[f'<b>{p:.2f}%</b>' for p in sorted_present],  # Add percentage labels for Present
        textposition='inside',  # Place text inside the bars for better readability
        insidetextanchor='end',  # Ensures text is well-placed for horizontal bars
    ))

    # Add Missing bars (horizontal orientation)
    fig.add_trace(go.Bar(
        y=sorted_properties,  # Set y-axis to properties (categories) for horizontal bars
        x=sorted_missing,
        name='Missing',
        orientation='h',  # Horizontal bars
        marker=dict(color='#00B1E2'),
        base=sorted_present,  # Stack missing on top of present
        text=[f'<b>{m:.2f}%</b>' for m in sorted_missing],  # Add percentage labels for Missing
        textposition='none',  # Hide text for Missing (optional)
    ))

    # Update layout for better readability and tighter spacing
    fig.update_layout(
        barmode='stack',
        title='Metadata Completeness by Property (Horizontal, Sorted by Present)',
        xaxis_title='Percentage',
        #yaxis_title='Property',
        font=dict(size=16),  # Reduced font size slightly for cleaner look
        hovermode='y unified',
        width=900,  # Slightly reduced width
        height=500,  # Reduced height to compress the chart vertically
        plot_bgcolor='white',  # Set plot background to white
        paper_bgcolor='white',  # Set entire figure background to white
        bargap=0.1,  # Almost zero gap between bars of different categories
        margin=dict(l=80, r=20, t=40, b=40),  # Adjust margins to reduce empty space further
        showlegend=True,  # Legend for clarity
    )

    # Show the interactive plotly figure
    fig.show()

# Main script
if __name__ == "__main__":
    # Google Sheets ID and sheet name
    sheet_id = '1PQe813L2M-aR_5i-1gEbHtP4M4iclgdedhdrj9DuhNg'
    sheet_name = 'Properties'

    # Load the data from Google Sheets
    df = load_data(sheet_id, sheet_name)

    # Plot the stacked horizontal bar chart
    plot_stacked_bar_chart(df)

## Recommended properties only

In [8]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Function to load the data from Google Sheets
def load_data(sheet_id, sheet_name):
    """Loads the data from the Google Sheets."""
    url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
    df = pd.read_csv(url)

    # Set the first column (Properties) as the index
    df = df.set_index(df.columns[0])

    # Slice the dataframe to include only columns A to T (i.e., first 5 columns)
    df = df.iloc[:, 5:11]

    # Convert the data to floats and multiply by 100 to get percentages
    df = df.apply(lambda col: col.map(lambda x: float(x) * 100 if pd.notnull(x) else 0))

    return df

# Function to plot the stacked bar chart
def plot_stacked_bar_chart(df):
    """Plots a horizontal stacked bar chart of Present vs Missing for each property."""
    # Extract the 'Present' and 'Missing' rows
    present = df.loc['Present'].values[0]  # Use only the first row for Present values
    missing = df.loc['Missing'].values[0]  # Use only the first row for Missing values
    properties = df.columns.tolist()  # List of properties

    # Create a DataFrame for sorting
    df_sorted = pd.DataFrame({
        'Properties': properties,
        'Present': present,
        'Missing': missing
    })

    # Sort the DataFrame by 'Present' in descending order
    df_sorted = df_sorted.sort_values(by='Present', ascending=True)

    # Extract the sorted properties, present, and missing values
    sorted_properties = df_sorted['Properties'].values
    sorted_present = df_sorted['Present'].values
    sorted_missing = df_sorted['Missing'].values

    # Create the stacked horizontal bar chart with Plotly
    fig = go.Figure()

    # Add Present bars (horizontal orientation)
    fig.add_trace(go.Bar(
        y=sorted_properties,  # Set y-axis to properties (categories) for horizontal bars
        x=sorted_present,
        name='Present',
        orientation='h',  # Horizontal bars
        marker=dict(color='#243b54'),
        text=[f'<b>{p:.2f}%</b>' for p in sorted_present],  # Add percentage labels for Present
        textposition='inside',  # Place text inside the bars for better readability
        insidetextanchor='end',  # Ensures text is well-placed for horizontal bars
    ))

    # Add Missing bars (horizontal orientation)
    fig.add_trace(go.Bar(
        y=sorted_properties,  # Set y-axis to properties (categories) for horizontal bars
        x=sorted_missing,
        name='Missing',
        orientation='h',  # Horizontal bars
        marker=dict(color='#00B1E2'),
        base=sorted_present,  # Stack missing on top of present
        text=[f'<b>{m:.2f}%</b>' for m in sorted_missing],  # Add percentage labels for Missing
        textposition='none',  # Hide text for Missing (optional)
    ))

    # Update layout for better readability and tighter spacing
    fig.update_layout(
        barmode='stack',
        title='Metadata Completeness by Property (Horizontal, Sorted by Present)',
        xaxis_title='Percentage',
        #yaxis_title='Property',
        font=dict(size=16),  # Reduced font size slightly for cleaner look
        hovermode='y unified',
        width=900,  # Slightly reduced width
        height=500,  # Reduced height to compress the chart vertically
        plot_bgcolor='white',  # Set plot background to white
        paper_bgcolor='white',  # Set entire figure background to white
        bargap=0.1,  # Almost zero gap between bars of different categories
        margin=dict(l=80, r=20, t=40, b=40),  # Adjust margins to reduce empty space further
        showlegend=True,  # Legend for clarity
    )

    # Show the interactive plotly figure
    fig.show()

# Main script
if __name__ == "__main__":
    # Google Sheets ID and sheet name
    sheet_id = '1PQe813L2M-aR_5i-1gEbHtP4M4iclgdedhdrj9DuhNg'
    sheet_name = 'Properties'

    # Load the data from Google Sheets
    df = load_data(sheet_id, sheet_name)

    # Plot the stacked horizontal bar chart
    plot_stacked_bar_chart(df)

## Optional Properties only

In [9]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Function to load the data from Google Sheets
def load_data(sheet_id, sheet_name):
    """Loads the data from the Google Sheets."""
    url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
    df = pd.read_csv(url)

    # Set the first column (Properties) as the index
    df = df.set_index(df.columns[0])

    # Slice the dataframe to include only columns A to T (i.e., first 5 columns)
    df = df.iloc[:, 11:19]

    # Convert the data to floats and multiply by 100 to get percentages
    df = df.apply(lambda col: col.map(lambda x: float(x) * 100 if pd.notnull(x) else 0))

    return df

# Function to plot the stacked bar chart
def plot_stacked_bar_chart(df):
    """Plots a horizontal stacked bar chart of Present vs Missing for each property."""
    # Extract the 'Present' and 'Missing' rows
    present = df.loc['Present'].values[0]  # Use only the first row for Present values
    missing = df.loc['Missing'].values[0]  # Use only the first row for Missing values
    properties = df.columns.tolist()  # List of properties

    # Create a DataFrame for sorting
    df_sorted = pd.DataFrame({
        'Properties': properties,
        'Present': present,
        'Missing': missing
    })

    # Sort the DataFrame by 'Present' in descending order
    df_sorted = df_sorted.sort_values(by='Present', ascending=True)

    # Extract the sorted properties, present, and missing values
    sorted_properties = df_sorted['Properties'].values
    sorted_present = df_sorted['Present'].values
    sorted_missing = df_sorted['Missing'].values

    # Create the stacked horizontal bar chart with Plotly
    fig = go.Figure()

    # Add Present bars (horizontal orientation)
    fig.add_trace(go.Bar(
        y=sorted_properties,  # Set y-axis to properties (categories) for horizontal bars
        x=sorted_present,
        name='Present',
        orientation='h',  # Horizontal bars
        marker=dict(color='#243b54'),
        text=[f'<b>{p:.2f}%</b>' for p in sorted_present],  # Add percentage labels for Present
        textposition='inside',  # Place text inside the bars for better readability
        insidetextanchor='end',  # Ensures text is well-placed for horizontal bars
    ))

    # Add Missing bars (horizontal orientation)
    fig.add_trace(go.Bar(
        y=sorted_properties,  # Set y-axis to properties (categories) for horizontal bars
        x=sorted_missing,
        name='Missing',
        orientation='h',  # Horizontal bars
        marker=dict(color='#00B1E2'),
        base=sorted_present,  # Stack missing on top of present
        text=[f'<b>{m:.2f}%</b>' for m in sorted_missing],  # Add percentage labels for Missing
        textposition='none',  # Hide text for Missing (optional)
    ))

    # Update layout for better readability and tighter spacing
    fig.update_layout(
        barmode='stack',
        title='Metadata Completeness by Property (Horizontal, Sorted by Present)',
        xaxis_title='Percentage',
        #yaxis_title='Property',
        font=dict(size=16),  # Reduced font size slightly for cleaner look
        hovermode='y unified',
        width=900,  # Slightly reduced width
        height=500,  # Reduced height to compress the chart vertically
        plot_bgcolor='white',  # Set plot background to white
        paper_bgcolor='white',  # Set entire figure background to white
        bargap=0.1,  # Almost zero gap between bars of different categories
        margin=dict(l=80, r=20, t=40, b=40),  # Adjust margins to reduce empty space further
        showlegend=True,  # Legend for clarity
    )

    # Show the interactive plotly figure
    fig.show()

# Main script
if __name__ == "__main__":
    # Google Sheets ID and sheet name
    sheet_id = '1PQe813L2M-aR_5i-1gEbHtP4M4iclgdedhdrj9DuhNg'
    sheet_name = 'Properties'

    # Load the data from Google Sheets
    df = load_data(sheet_id, sheet_name)

    # Plot the stacked horizontal bar chart
    plot_stacked_bar_chart(df)

## Top 200 providers over time

In [10]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Function to clean and format the data
def clean_data(df):
    """Cleans and formats the input DataFrame for plotting."""
    # Ensure "Cumulative Sum of DOIs registered" column (index 2) is numeric, remove commas if necessary
    df.iloc[:, 2] = pd.to_numeric(df.iloc[:, 2].replace({',': ''}, regex=True), errors='coerce')

    # Cast years to integers (to avoid floating point display like 2022.0)
    df.iloc[:, 1] = df.iloc[:, 1].astype(int)

    # Fill any missing values with 0 (providers that don't have data for certain years)
    return df.fillna(0)

# Function to plot a stacked bar chart
def plot_stacked_bar_chart(data_pivot):
    """Plots a stacked bar chart for DOI data per provider over years."""
    colors = ['#243b54', '#8ed1fc', '#0693e3', '#0ea0d7']  # Define colors

    # Create the figure
    fig = go.Figure()

    # Initialize bottom_data to stack the bars
    bottom_data = np.zeros(len(data_pivot))

    # Loop through each provider and add bars
    for idx, provider_id in enumerate(data_pivot.columns):
        fig.add_trace(go.Bar(
            x=data_pivot.index,
            y=data_pivot[provider_id],
            name=provider_id,
            marker=dict(color=colors[idx % len(colors)]),
            text=[f'{int(v):,}' if v > 400000 else '' for v in data_pivot[provider_id]],
            textposition='outside',
            hovertemplate=(
                f'<b>Provider:</b> {provider_id}<br>'
                '<b>Year:</b> %{x}<br>'
                '<b>DOIs Registered:</b> %{y:,}<extra></extra>'
            ),  # Custom hover text to display provider, year, and DOIs
        ))

    # Update layout for better readability
    fig.update_layout(
        barmode='stack',
        title='Cumulative DOIs Registered per Provider Over Years',
        xaxis_title='Year',
        yaxis_title='Cumulative Sum of DOIs Registered',
        font=dict(size=14),
        hovermode='closest',
        xaxis_tickangle=-45,  # Rotate x-axis labels
        width=1800,  # Set figure width
        height=600,   # Set figure height
        plot_bgcolor='white',  # Set plot background to white
        paper_bgcolor='white',  # Set entire figure background to white
        showlegend=True  # Enable the legend box
    )

    # Update x-axis and y-axis to set axis lines and ticks to light grey
    fig.update_xaxes(
        showline=True,  # Show x-axis line
        linecolor='lightgrey',  # Set x-axis line color to light grey
        showgrid=False  # Optionally remove grid lines for x-axis
    )

    fig.update_yaxes(
        showline=True,  # Show y-axis line
        linecolor='lightgrey',  # Set y-axis line color to light grey
        tickformat=',',  # Add commas to large numbers
        showgrid=False  # Optionally remove grid lines for y-axis
    )

    # Update axes ticks and labels
    fig.update_xaxes(
        tickvals=data_pivot.index,
        ticktext=data_pivot.index.astype(int),
        tickfont=dict(size=20)
    )

    fig.update_yaxes(
        tickformat=',',  # Show y-axis values with commas for large numbers
        tickfont=dict(size=20)
    )

    # Add a legend outside of the chart area
    fig.update_layout(
        legend=dict(title='Providers', x=1, y=1, traceorder='normal', xanchor='left')
    )

    # Show the interactive plotly figure
    fig.show()

# Main script
if __name__ == "__main__":
    # Google Sheets ID and sheet name
    sheet_id = '1PQe813L2M-aR_5i-1gEbHtP4M4iclgdedhdrj9DuhNg'
    sheet_name = 'DOIs_Providers'
    url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

    # Load the spreadsheet
    spreadsheet = pd.read_csv(url)

    # Clean the data
    spreadsheet = clean_data(spreadsheet)

    # Pivot the data using column locations: index 1 (Year), columns 0 (provider_id_and_name), and values 2 (Cumulative DOIs)
    data_pivot = pd.pivot_table(spreadsheet, values=spreadsheet.columns[2],
                                index=spreadsheet.columns[1], columns=spreadsheet.columns[0], aggfunc=np.sum)

    # Fill any missing values with 0
    data_pivot = data_pivot.fillna(0)

    # Plot the stacked bar chart
    plot_stacked_bar_chart(data_pivot)

<ipython-input-10-979031b45c9d>:108: FutureWarning:

The provided callable <function sum at 0x79bf644ab370> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.



## Resource Type General (controlled vocabulary)

In [11]:
import pandas as pd
import plotly.graph_objects as go

# Load the data from the Google Sheet
sheet_url = "https://docs.google.com/spreadsheets/d/1PQe813L2M-aR_5i-1gEbHtP4M4iclgdedhdrj9DuhNg/gviz/tq?tqx=out:csv&sheet=rTGs"
df = pd.read_csv(sheet_url)

# Ensure no missing values, and convert the 'Count' column (Column B: index 1) to numeric
df.iloc[:, 1] = pd.to_numeric(df.iloc[:, 1], errors='coerce').fillna(0)

# Sort the data by count (Column B: index 1) in descending order
df = df.sort_values(df.columns[1], ascending=False).reset_index(drop=True)

# Define a muted color palette for the top 10, and grey for the rest
colors = ['#2a4d69', '#00B1E2', '#adcbe3', '#63ace5', '#ee4035', '#f37736', '#fdf498', '#7bc043', '#0392cf', '#ee7e77'] + ['#cccccc'] * (len(df) - 10)

# Create hover text for all categories, even the small ones
hover_text = [f'{df.iloc[i, 0]}: {df.iloc[i, 1]:,.0f}' for i in range(len(df))]

# Plotly figure for an interactive doughnut chart
fig = go.Figure(go.Pie(
    labels=df.iloc[:, 0],  # Column A for resource types
    values=df.iloc[:, 1],  # Column B for counts
    hoverinfo='label+percent+value',  # Display label, percentage, and actual value on hover
    textinfo='value+label',  # No text displayed on the slices
    hole=0.4,  # Doughnut hole size
    textposition='inside',  # Position the text inside the slices by default
    insidetextorientation='radial',  # Radial text orientation for better readability
    insidetextfont=dict(size=14, color='white'),  # Smaller font for text inside slices
    marker=dict(colors=colors, line=dict(color='#FFFFFF', width=2))  # Slice colors and white borders
))

# Update layout to add legend and ensure the chart is a circle
fig.update_layout(
    title_text="Resource Type General Distribution",
    annotations=[dict(text='Resources', x=0.5, y=0.5, font_size=20, showarrow=False)],
    showlegend=True,  # Enable the legend to show all categories
    legend=dict(yanchor="top", y=1, xanchor="left", x=1.5),  # Place legend outside the chart
    margin=dict(l=50, r=50, t=50, b=50),  # Adjust margins for better visualization
    height=1200,  # Adjust height of the chart
    width=1500,
    plot_bgcolor='white',  # Set plot background to white
    paper_bgcolor='white',  # Set entire figure background to white
)

# Display the figure
fig.show()


## Resource Types (Free text)

In [12]:
import pandas as pd
import plotly.graph_objects as go

# Load the data from the Google Sheet
sheet_url = "https://docs.google.com/spreadsheets/d/1PQe813L2M-aR_5i-1gEbHtP4M4iclgdedhdrj9DuhNg/gviz/tq?tqx=out:csv&sheet=rTs"
df = pd.read_csv(sheet_url)

# Ensure no missing values, and convert the 'Count' column (Column B: index 1) to numeric
df.iloc[:, 1] = pd.to_numeric(df.iloc[:, 1], errors='coerce').fillna(0)

# Sort the data by count (Column B: index 1) in descending order
df = df.sort_values(df.columns[1], ascending=False).reset_index(drop=True)

# Define a muted color palette for the top 10, and grey for the rest
colors = ['#2a4d69', '#4b86b4', '#adcbe3', '#63ace5', '#ee4035', '#f37736', '#fdf498', '#7bc043', '#0392cf', '#ee7e77'] + ['#cccccc'] * (len(df) - 10)

# Create hover text for all categories, even the small ones
hover_text = [f'{df.iloc[i, 0]}: {df.iloc[i, 1]:,.0f}' for i in range(len(df))]

# Plotly figure for an interactive doughnut chart
fig = go.Figure(go.Pie(
    labels=df.iloc[:, 0],  # Column A for resource types
    values=df.iloc[:, 1],  # Column B for counts
    hoverinfo='label+percent+value',  # Display label, percentage, and actual value on hover
    textinfo='value+label',  # No text displayed on the slices
    hole=0.4,  # Doughnut hole size
    textposition='inside',  # Position the text inside the slices by default
    insidetextorientation='radial',  # Radial text orientation for better readability
    insidetextfont=dict(size=14, color='white'),  # Smaller font for text inside slices
    marker=dict(colors=colors, line=dict(color='#FFFFFF', width=2))  # Slice colors and white borders
))

# Update layout to add legend and ensure the chart is a circle
fig.update_layout(
    title_text="Resource Type General Distribution",
    annotations=[dict(text='Resources', x=0.5, y=0.5, font_size=20, showarrow=False)],
    showlegend=True,  # Enable the legend to show all categories
    legend=dict(yanchor="top", y=1, xanchor="left", x=1.5),  # Place legend outside the chart
    margin=dict(l=50, r=50, t=50, b=50),  # Adjust margins for better visualization
    height=1200,  # Adjust height of the chart
    width=1500
)

# Display the figure
fig.show()


## Connections

In [13]:
import pandas as pd
import plotly.graph_objects as go

# The Google Sheets URL for your DOIs_OT sheet
sheet_id = '1PQe813L2M-aR_5i-1gEbHtP4M4iclgdedhdrj9DuhNg'
sheet_name = 'Connections'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

# Load the spreadsheet
spreadsheet = pd.read_csv(url)

# Ensure the "Years" column is standardized as string
spreadsheet.iloc[:, 0] = spreadsheet.iloc[:, 0].astype(str)

# Extract the relevant columns by index location (0 = years, 1 = cumulative sum)
years = spreadsheet.iloc[:, 0]  # Column 0: "Years"
cumulative_sum = spreadsheet.iloc[:, 1].replace({',': ''}, regex=True).astype(int)  # Column 1: "Cumulative Sum of DOIs registered"

# Handle potential duplicates by grouping and summing (still using the location index for columns)
spreadsheet = spreadsheet.groupby(spreadsheet.iloc[:, 0]).agg({spreadsheet.columns[1]: 'sum'}).reset_index()

# Sort the data by cumulative sum in descending order
sorted_spreadsheet = spreadsheet.sort_values(by=spreadsheet.columns[1], ascending=False)

# Create variables for the sorted values
years_sorted = sorted_spreadsheet.iloc[:, 0]  # Sorted years
cumulative_sum_sorted = sorted_spreadsheet.iloc[:, 1]  # Sorted cumulative sum

# Create the Plotly bar chart
fig = go.Figure()

# Plot the bars
fig.add_trace(go.Bar(
    x=years_sorted,
    y=cumulative_sum_sorted,
    text=[f'{int(val):,}' for val in cumulative_sum_sorted],  # Add commas for large numbers
    textposition='outside',  # Automatically place the text inside or outside the bars
    marker=dict(color='#2a4d69'),  # Define bar color
))

# Update layout for titles, labels, and formatting
fig.update_layout(
    title='Number of DOIs with Connection Metadata',
    xaxis_title='Connection fields',
    yaxis_title='Number of DOIs',
    font=dict(size=18),
    hovermode='closest',
    xaxis_tickangle=-45,  # Rotate x-axis labels for readability
    width=1500,  # Adjust figure width
    height=1000,   # Adjust figure height
    plot_bgcolor='white',  # Set plot background to white
    paper_bgcolor='white',  # Set entire figure background to white
)
# Update x-axis and y-axis to set axis lines and ticks to light grey
fig.update_xaxes(
    showline=True,  # Show x-axis line
    linecolor='lightgrey',  # Set x-axis line color to light grey
    showgrid=False  # Optionally remove grid lines for x-axis
    )

fig.update_yaxes(
    showline=True,  # Show y-axis line
    linecolor='lightgrey',  # Set y-axis line color to light grey
    tickformat=',',  # Add commas to large numbers
    showgrid=False  # Optionally remove grid lines for y-axis
    )
# Update y-axis formatting for large numbers with commas
fig.update_yaxes(tickformat=',')

# Show the Plotly chart
fig.show()

## Connections with ORCID

In [14]:
import pandas as pd
import plotly.graph_objects as go

# The Google Sheets URL for your DOIs_OT sheet
sheet_id = '1PQe813L2M-aR_5i-1gEbHtP4M4iclgdedhdrj9DuhNg'
sheet_name = 'ORCID'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

# Load the spreadsheet
spreadsheet = pd.read_csv(url)

# Clean the data and ensure "Years" column is standardized
# spreadsheet = spreadsheet.dropna()  # Uncomment if you want to drop missing values
spreadsheet.iloc[:, 0] = spreadsheet.iloc[:, 0].astype(str)  # Ensure the "Years" column is a string

# Extract the relevant columns
years = spreadsheet.iloc[:, 0]  # Column 0: "Years"
cumulative_sum = spreadsheet.iloc[:, 1].replace({',': ''}, regex=True).astype(int)  # Column 1: "Cumulative Sum of DOIs"

# Handle potential duplicates by grouping and summing
spreadsheet = spreadsheet.groupby(spreadsheet.iloc[:, 0]).agg({spreadsheet.columns[1]: 'sum'}).reset_index()

# Sort the data by cumulative sum in descending order
sorted_spreadsheet = spreadsheet.sort_values(by=spreadsheet.columns[1], ascending=False)

# Create variables for the sorted values
years_sorted = sorted_spreadsheet.iloc[:, 0]  # Sorted years
cumulative_sum_sorted = sorted_spreadsheet.iloc[:, 1]  # Sorted cumulative sum

# Create the Plotly bar chart
fig = go.Figure()

# Add bars to the chart
fig.add_trace(go.Bar(
    x=years_sorted,
    y=cumulative_sum_sorted,
    text=[f'{int(val):,}' for val in cumulative_sum_sorted],  # Add commas for large numbers
    textposition='outside',  # Position text inside or outside automatically
    marker=dict(color='#243b54'),  # Define bar color
))

# Update layout for titles and labels
fig.update_layout(
    title='Number of DOIs with Connection Metadata including ORCID',
    xaxis_title='Connection fields',
    yaxis_title='Number of DOIs',
    font=dict(size=18),
    hovermode='closest',
    xaxis_tickangle=-45,  # Rotate x-axis labels
    width=1200,  # Adjust figure width
    height=1000,   # Adjust figure height
    plot_bgcolor='white',  # Set plot background to white
    paper_bgcolor='white',  # Set entire figure background to white
)
# Update x-axis and y-axis to set axis lines and ticks to light grey
fig.update_xaxes(
    showline=True,  # Show x-axis line
    linecolor='lightgrey',  # Set x-axis line color to light grey
    showgrid=False  # Optionally remove grid lines for x-axis
    )

fig.update_yaxes(
    showline=True,  # Show y-axis line
    linecolor='lightgrey',  # Set y-axis line color to light grey
    tickformat=',',  # Add commas to large numbers
    showgrid=False  # Optionally remove grid lines for y-axis
    )
# Update y-axis formatting for large numbers
fig.update_yaxes(tickformat=',')  # Format with commas

# Display the Plotly chart
fig.show()

## Connections with ORCID Over Time

In [15]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Function to clean and format the data
def clean_data(df):
    """Cleans and formats the input DataFrame for plotting."""
    # Ensure the "Number of DOIs provided per repository" column (index 2) is numeric, remove commas if necessary
    df.iloc[:, 2] = pd.to_numeric(df.iloc[:, 2].replace({',': ''}, regex=True), errors='coerce')

    # Fill any missing values with 0
    return df.fillna(0)

# Function to plot a stacked bar chart
def plot_stacked_bar_chart(data_pivot):
    """Plots a stacked bar chart for DOI data per repository over funders."""
    # Define the color palette
    colors = ['#243b54', '#0693e3', '#0ea0d7', '#46BCAB', '#F07C73']

    # Create the Plotly figure
    fig = go.Figure()

    # Initialize bottom_data for stacking the bars
    bottom_data = np.zeros(len(data_pivot))

    # Loop through each repository and add bars
    for idx, repo_id in enumerate(data_pivot.columns):
        fig.add_trace(go.Bar(
            x=data_pivot.index,
            y=data_pivot[repo_id],
            name=repo_id,
            marker=dict(color=colors[idx % len(colors)]),
            text=[f'{int(val):,}' if val > 40000 else '' for val in data_pivot[repo_id]],  # Display value if > 40,000
            textposition='outside',  # Automatically place the text inside the bars
            hovertemplate=(
                f'<b>Connection metadata:</b> {repo_id}<br>' +
                '<b>Year:</b> %{x}<br>' +
                '<b>Number of DOIs:</b> %{y:,}<extra></extra>'
            )  # Add repository, year, and number of DOIs in the hover info
        ))

    # Update layout to adjust the chart
    fig.update_layout(
        barmode='stack',
        title='Number of DOIs with Connection Metadata including ORCID over time',
        xaxis_title='Years',
        yaxis_title='Number of DOIs',
        font=dict(size=18),
        hovermode='closest',
        xaxis_tickangle=-45,  # Rotate x-axis labels for readability
        width=1600,  # Set figure width
        height=1000,   # Set figure height
        plot_bgcolor='white',  # Set plot background to white
        paper_bgcolor='white',  # Set entire figure background to white
    )
    # Update x-axis and y-axis to set axis lines and ticks to light grey
    fig.update_xaxes(
        showline=True,  # Show x-axis line
        linecolor='lightgrey',  # Set x-axis line color to light grey
        showgrid=False  # Optionally remove grid lines for x-axis
        )

    fig.update_yaxes(
        showline=True,  # Show y-axis line
        linecolor='lightgrey',  # Set y-axis line color to light grey
        tickformat=',',  # Add commas to large numbers
        showgrid=False  # Optionally remove grid lines for y-axis
        )
    # Update x-axis ticks and labels
    fig.update_xaxes(
        tickvals=data_pivot.index,
        ticktext=data_pivot.index.astype(int),
        tickfont=dict(size=20)
    )

    # Update y-axis formatting to remove scientific notation
    fig.update_yaxes(tickformat=',', tickfont=dict(size=20))

    # Add legend and adjust its properties
    fig.update_layout(
        legend=dict(title='Connection Metadata', xanchor="left", yanchor="top", font=dict(size=14)),
        showlegend=True  # Show legend
    )

    # Show the interactive chart
    fig.show()

# Main script
if __name__ == "__main__":
    # Google Sheets ID and sheet name
    sheet_id = '1PQe813L2M-aR_5i-1gEbHtP4M4iclgdedhdrj9DuhNg'
    sheet_name = 'ORCID_OT'
    url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

    # Load the spreadsheet
    spreadsheet = pd.read_csv(url)

    # Clean the data
    spreadsheet = clean_data(spreadsheet)

    # Pivot the data using column locations: index 0 (Funder name), columns 1 (Repository name), and values 2 (Number of DOIs provided)
    data_pivot = pd.pivot_table(spreadsheet, values=spreadsheet.columns[2],
                                index=spreadsheet.columns[0], columns=spreadsheet.columns[1], aggfunc=np.sum)

    # Fill any missing values with 0
    data_pivot = data_pivot.fillna(0)

    # Sort the pivot table by the total number of DOIs per funder in descending order
    data_pivot['Total_DOIs'] = data_pivot.sum(axis=1)  # Add a new column for total DOIs per funder
    data_pivot = data_pivot.sort_values(by='Total_DOIs', ascending=False)  # Sort by total DOIs in descending order
    data_pivot = data_pivot.drop(columns=['Total_DOIs'])  # Remove the temporary total column after sorting

    # Plot the stacked bar chart
    plot_stacked_bar_chart(data_pivot)

<ipython-input-15-220812f6967f>:102: FutureWarning:

The provided callable <function sum at 0x79bf644ab370> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.



## Connections with ROR

In [16]:
import pandas as pd
import plotly.graph_objects as go

# The Google Sheets URL for your DOIs_OT sheet
sheet_id = '1PQe813L2M-aR_5i-1gEbHtP4M4iclgdedhdrj9DuhNg'
sheet_name = 'ROR'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

# Load the spreadsheet
spreadsheet = pd.read_csv(url)

# Ensure the "Years" column is standardized as string
spreadsheet.iloc[:, 0] = spreadsheet.iloc[:, 0].astype(str)

# Extract the relevant columns by index location (0 = years, 1 = cumulative sum)
years = spreadsheet.iloc[:, 0]  # Column 0: "Years"
cumulative_sum = spreadsheet.iloc[:, 1].replace({',': ''}, regex=True).astype(int)  # Column 1: "Cumulative Sum of DOIs registered"

# Handle potential duplicates by grouping and summing
spreadsheet = spreadsheet.groupby(spreadsheet.iloc[:, 0]).agg({spreadsheet.columns[1]: 'sum'}).reset_index()

# Sort the data by cumulative sum in descending order
sorted_spreadsheet = spreadsheet.sort_values(by=spreadsheet.columns[1], ascending=False)

# Create variables for the sorted values
years_sorted = sorted_spreadsheet.iloc[:, 0]  # Sorted years
cumulative_sum_sorted = sorted_spreadsheet.iloc[:, 1]  # Sorted cumulative sum

# Create the Plotly bar chart
fig = go.Figure()

# Plot the bars
fig.add_trace(go.Bar(
    x=years_sorted,
    y=cumulative_sum_sorted,
    text=[f'{int(val):,}' for val in cumulative_sum_sorted],  # Add commas for large numbers
    textposition='outside',  # Automatically place the text inside or outside the bars
    marker=dict(color='#243b54'),  # Define bar color
))

# Update layout for titles, labels, and formatting
fig.update_layout(
    title='Number of DOIs with Connection Metadata including ROR',
    xaxis_title='Connection fields',
    yaxis_title='Number of DOIs',
    font=dict(size=18),
    hovermode='closest',
    xaxis_tickangle=-45,  # Rotate x-axis labels for readability
    width=2000,  # Adjust figure width
    height=1000,   # Adjust figure height
    plot_bgcolor='white',  # Set plot background to white
    paper_bgcolor='white',  # Set entire figure background to white
)
    # Update x-axis and y-axis to set axis lines and ticks to light grey
fig.update_xaxes(
    showline=True,  # Show x-axis line
    linecolor='lightgrey',  # Set x-axis line color to light grey
    showgrid=False  # Optionally remove grid lines for x-axis
        )

fig.update_yaxes(
    showline=True,  # Show y-axis line
    linecolor='lightgrey',  # Set y-axis line color to light grey
    tickformat=',',  # Add commas to large numbers
    showgrid=False  # Optionally remove grid lines for y-axis
        )
# Update y-axis formatting for large numbers with commas
fig.update_yaxes(tickformat=',')

# Show the Plotly chart
fig.show()

## Connections with ROR Over Time

In [17]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Function to clean and format the data
def clean_data(df):
    """Cleans and formats the input DataFrame for plotting."""
    # Ensure the "Number of DOIs provided per repository" column (index 2) is numeric, remove commas if necessary
    df.iloc[:, 2] = pd.to_numeric(df.iloc[:, 2].replace({',': ''}, regex=True), errors='coerce')

    # Fill any missing values with 0
    return df.fillna(0)

# Function to plot a stacked bar chart
def plot_stacked_bar_chart(data_pivot):
    """Plots a stacked bar chart for DOI data per repository over funders."""
    # Define the color palette
    colors = ['#243b54', '#0693e3', '#0ea0d7', '#46BCAB', '#F07C73']

    # Create the Plotly figure
    fig = go.Figure()

    # Initialize bottom_data for stacking the bars
    bottom_data = np.zeros(len(data_pivot))

    # Loop through each repository and add bars
    for idx, repo_id in enumerate(data_pivot.columns):
        fig.add_trace(go.Bar(
            x=data_pivot.index,
            y=data_pivot[repo_id],
            name=repo_id,
            marker=dict(color=colors[idx % len(colors)]),
            text=[f'{int(val):,}' if val > 40000 else '' for val in data_pivot[repo_id]],  # Display value if > 40,000
            textposition='outside',  # Automatically place the text inside the bars
            hovertemplate=(
                f'<b>Connection metadata:</b> {repo_id}<br>' +
                '<b>Year:</b> %{x}<br>' +
                '<b>Number of DOIs:</b> %{y:,}<extra></extra>'
            )  # Add repository, year, and number of DOIs in the hover info
        ))

    # Update layout for better readability
    fig.update_layout(
        barmode='stack',
        title='Number of DOIs with Connection Metadata including ROR over time',
        xaxis_title='Years',
        yaxis_title='Number of DOIs',
        font=dict(size=18),
        hovermode='closest',
        xaxis_tickangle=-45,  # Rotate x-axis labels for readability
        width=2000,  # Set figure width
        height=1000,   # Set figure height
        plot_bgcolor='white',  # Set plot background to white
        paper_bgcolor='white',  # Set entire figure background to white
    )
    # Update x-axis and y-axis to set axis lines and ticks to light grey
    fig.update_xaxes(
        showline=True,  # Show x-axis line
        linecolor='lightgrey',  # Set x-axis line color to light grey
        showgrid=False  # Optionally remove grid lines for x-axis
        )

    fig.update_yaxes(
        showline=True,  # Show y-axis line
        linecolor='lightgrey',  # Set y-axis line color to light grey
        tickformat=',',  # Add commas to large numbers
        showgrid=False  # Optionally remove grid lines for y-axis
        )
    # Update x-axis ticks and labels
    fig.update_xaxes(
        tickvals=data_pivot.index,
        ticktext=data_pivot.index.astype(int),
        tickfont=dict(size=20)
    )

    # Update y-axis formatting to remove scientific notation and use commas
    fig.update_yaxes(tickformat=',', tickfont=dict(size=20))

    # Add legend and adjust its properties
    fig.update_layout(
        legend=dict(title='Connection Metadata', xanchor="left", yanchor="top", font=dict(size=14)),
        showlegend=True  # Show legend
    )

    # Show the interactive chart
    fig.show()

# Main script
if __name__ == "__main__":
    # Google Sheets ID and sheet name
    sheet_id = '1PQe813L2M-aR_5i-1gEbHtP4M4iclgdedhdrj9DuhNg'
    sheet_name = 'ROR_OT'
    url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

    # Load the spreadsheet
    spreadsheet = pd.read_csv(url)

    # Clean the data
    spreadsheet = clean_data(spreadsheet)

    # Pivot the data using column locations: index 0 (Funder name), columns 1 (Repository name), and values 2 (Number of DOIs provided)
    data_pivot = pd.pivot_table(spreadsheet, values=spreadsheet.columns[2],
                                index=spreadsheet.columns[0], columns=spreadsheet.columns[1], aggfunc=np.sum)

    # Fill any missing values with 0
    data_pivot = data_pivot.fillna(0)

    # Sort the pivot table by the total number of DOIs per funder in descending order
    data_pivot['Total_DOIs'] = data_pivot.sum(axis=1)  # Add a new column for total DOIs per funder
    data_pivot = data_pivot.sort_values(by='Total_DOIs', ascending=False)  # Sort by total DOIs in descending order
    data_pivot = data_pivot.drop(columns=['Total_DOIs'])  # Remove the temporary total column after sorting

    # Plot the stacked bar chart
    plot_stacked_bar_chart(data_pivot)

<ipython-input-17-94dce2df2b17>:102: FutureWarning:

The provided callable <function sum at 0x79bf644ab370> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.



## Top Ten Funders

In [18]:
import pandas as pd
import plotly.graph_objects as go

# The Google Sheets URL for your DOIs_OT sheet
sheet_id = '1PQe813L2M-aR_5i-1gEbHtP4M4iclgdedhdrj9DuhNg'
sheet_name = 'FunderInfo'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

# Load the spreadsheet
spreadsheet = pd.read_csv(url)

# Clean the data if necessary (dropping missing values is optional)
# spreadsheet = spreadsheet.dropna()

# Ensure the "Years" column is standardized as string
spreadsheet.iloc[:, 0] = spreadsheet.iloc[:, 0].astype(str)

# Extract the relevant columns (years and cumulative sum)
years = spreadsheet.iloc[:, 0]  # Column 0: "Years"
cumulative_sum = spreadsheet.iloc[:, 1].replace({',': ''}, regex=True).astype(int)  # Column 1: "Cumulative Sum of DOIs registered"

# Group by years and sum the DOIs
spreadsheet = spreadsheet.groupby(spreadsheet.iloc[:, 0]).agg({spreadsheet.columns[1]: 'sum'}).reset_index()

# Sort the data by cumulative sum in descending order
sorted_spreadsheet = spreadsheet.sort_values(by=spreadsheet.columns[1], ascending=False)

# Create variables for the sorted values
years_sorted = sorted_spreadsheet.iloc[:, 0]  # Sorted years
cumulative_sum_sorted = sorted_spreadsheet.iloc[:, 1]  # Sorted cumulative sum

# Create the Plotly bar chart
fig = go.Figure()

# Plot the bars
fig.add_trace(go.Bar(
    x=years_sorted,
    y=cumulative_sum_sorted,
    text=[f'{int(val):,}' for val in cumulative_sum_sorted],  # Add commas for large numbers
    textposition='outside',  # Automatically place the text inside or outside the bars
    marker=dict(color='#243b54'),  # Define bar color
))

# Update layout for titles, labels, and formatting
fig.update_layout(
    title='Number of DOIs with Funders Name',
    xaxis_title='Funders Names',
    yaxis_title='Number of DOIs',
    font=dict(size=18),
    hovermode='closest',
    xaxis_tickangle=-45,  # Rotate x-axis labels for readability
    width=2000,  # Adjust figure width
    height=1000,   # Adjust figure height
    plot_bgcolor='white',  # Set plot background to white
    paper_bgcolor='white',  # Set entire figure background to white
)
# Update x-axis and y-axis to set axis lines and ticks to light grey
fig.update_xaxes(
    showline=True,  # Show x-axis line
    linecolor='lightgrey',  # Set x-axis line color to light grey
    showgrid=False  # Optionally remove grid lines for x-axis
)

fig.update_yaxes(
    showline=True,  # Show y-axis line
    linecolor='lightgrey',  # Set y-axis line color to light grey
    tickformat=',',  # Add commas to large numbers
    showgrid=False  # Optionally remove grid lines for y-axis
)

# Update y-axis formatting for large numbers with commas
fig.update_yaxes(tickformat=',')

# Show the Plotly chart
fig.show()

## Top Ten Repositories contributing to funders information

In [19]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Function to clean and format the data
def clean_data(df):
    """Cleans and formats the input DataFrame for plotting."""
    # Ensure the "Number of DOIs provided per repository" column (index 2) is numeric, remove commas if necessary
    df.iloc[:, 2] = pd.to_numeric(df.iloc[:, 2].replace({',': ''}, regex=True), errors='coerce')

    # Fill any missing values with 0
    return df.fillna(0)

# Function to plot a stacked bar chart
def plot_stacked_bar_chart(data_pivot):
    """Plots a stacked bar chart for DOI data per repository over funders."""
    # Define the color palette
    colors = ['#243b54', '#0693e3', '#C0CED6', '#46BCAB', '#F07C73']

    # Create the Plotly figure
    fig = go.Figure()

    # Initialize bottom_data for stacking the bars
    bottom_data = np.zeros(len(data_pivot))

    # Loop through each repository and add bars
    for idx, repo_id in enumerate(data_pivot.columns):
        total_contribution = data_pivot[repo_id].sum()

        # Only show repositories in the legend if they contributed more than 40,000 DOIs
        show_legend = bool(total_contribution > 40000)

        fig.add_trace(go.Bar(
            x=data_pivot.index,
            y=data_pivot[repo_id],
            name=repo_id if show_legend else '',  # Only show in legend if contribution > 40,000
            marker=dict(color=colors[idx % len(colors)]),
            text=[f'{int(val):,}' if val > 40000 else '' for val in data_pivot[repo_id]],  # Display value if > 40,000
            textposition='outside',  # Automatically place the text inside or outside the bars

            showlegend=show_legend,  # Show in legend only if contribution > 40,000
            hovertemplate=(
                f'<b>Repository:</b> {repo_id}<br>'  # Show repository name (metadata point)
                '<b>Year:</b> %{x}<br>'  # Show year
                '<b>DOIs Provided:</b> %{y:,}<br>'  # Show number of DOIs
                '<extra></extra>'  # This suppresses extra hover info
            )
        ))

    # Update layout for better readability
    fig.update_layout(
        barmode='stack',
        title='DOIs Provided per Repository for Each Funder',
        xaxis_title='Funders',
        yaxis_title='Number of DOIs Provided',
        font=dict(size=18),
        hovermode='closest',
        xaxis_tickangle=-45,  # Rotate x-axis labels for readability
        width=2000,  # Set figure width
        height=1000,   # Set figure height
        plot_bgcolor='white',  # Set plot background to white
        paper_bgcolor='white',  # Set entire figure background to white
    )
          # Update x-axis and y-axis to set axis lines and ticks to light grey
    fig.update_xaxes(
        showline=True,  # Show x-axis line
        linecolor='lightgrey',  # Set x-axis line color to light grey
        showgrid=False  # Optionally remove grid lines for x-axis
    )

    fig.update_yaxes(
        showline=True,  # Show y-axis line
        linecolor='lightgrey',  # Set y-axis line color to light grey
        tickformat=',',  # Add commas to large numbers
        showgrid=False  # Optionally remove grid lines for y-axis
    )

    # Update x-axis ticks and labels
    fig.update_xaxes(
        tickvals=data_pivot.index,
        ticktext=data_pivot.index.astype(str),
        tickfont=dict(size=20)
    )

    # Update y-axis formatting to remove scientific notation and use commas
    fig.update_yaxes(tickformat=',', tickfont=dict(size=20))

    # Adjust the legend position
    fig.update_layout(
        legend=dict(title='Repositories', xanchor="auto", yanchor="top", font=dict(size=14)),
        showlegend=True  # Show filtered legend
    )

    # Show the interactive chart
    fig.show()

# Main script
if __name__ == "__main__":
    # Google Sheets ID and sheet name
    sheet_id = '1PQe813L2M-aR_5i-1gEbHtP4M4iclgdedhdrj9DuhNg'
    sheet_name = 'Funder_repos'
    url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

    # Load the spreadsheet
    spreadsheet = pd.read_csv(url)

    # Clean the data
    spreadsheet = clean_data(spreadsheet)

    # Pivot the data using column locations: index 0 (Funder name), columns 1 (Repository name), and values 2 (Number of DOIs provided)
    data_pivot = pd.pivot_table(spreadsheet, values=spreadsheet.columns[2],
                                index=spreadsheet.columns[0], columns=spreadsheet.columns[1], aggfunc=np.sum)

    # Fill any missing values with 0
    data_pivot = data_pivot.fillna(0)

    # Sort the pivot table by the total number of DOIs per funder in descending order
    data_pivot['Total_DOIs'] = data_pivot.sum(axis=1)  # Add a new column for total DOIs per funder
    data_pivot = data_pivot.sort_values(by='Total_DOIs', ascending=False)  # Sort by total DOIs in descending order
    data_pivot = data_pivot.drop(columns=['Total_DOIs'])  # Remove the temporary total column after sorting

    # Plot the stacked bar chart
    plot_stacked_bar_chart(data_pivot)

<ipython-input-19-2263312604e4>:111: FutureWarning:

The provided callable <function sum at 0x79bf644ab370> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.



## Funders and associated creator & contributor affiliations

In [20]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Function to clean and format the data
def clean_data(df):
    """Cleans and formats the input DataFrame for plotting."""
    # Ensure the "Number of DOIs provided per repository" column (index 2) is numeric, remove commas if necessary
    df.iloc[:, 2] = pd.to_numeric(df.iloc[:, 2].replace({',': ''}, regex=True), errors='coerce')

    # Fill any missing values with 0
    return df.fillna(0)

# Function to plot a stacked bar chart
def plot_stacked_bar_chart(data_pivot):
    """Plots a stacked bar chart for DOI data per repository over funders."""
    # Define the color palette
    colors = ['#243b54', '#C0CED6','#46BCAB', '#0693e3', '#00B1E2', '#F07C73']

    # Create the Plotly figure
    fig = go.Figure()

    # Initialize bottom_data for stacking the bars
    bottom_data = np.zeros(len(data_pivot))

    # Loop through each repository and add bars
    for idx, repo_id in enumerate(data_pivot.columns):
        total_contribution = data_pivot[repo_id].sum()

        # Only show repositories in the legend if they contributed more than 40,000 DOIs
        show_legend = bool(total_contribution > 40000)

        # Add bars to the chart
        fig.add_trace(go.Bar(
            x=data_pivot.index,
            y=data_pivot[repo_id],
            name=repo_id if show_legend else '',  # Only show in legend if contribution > 40,000
            marker=dict(color=colors[(idx + 2) % len(colors)]),
            text=[f'{int(val):,}' if val > 40000 else '' for val in data_pivot[repo_id]],  # Display value if > 40,000
            textposition='auto',  # Automatically place the text
            textangle=45,  # Rotate the text by 45 degrees
            hoverinfo='x+y',
            showlegend=show_legend  # Show in legend only if contribution > 40,000
        ))

        # Add repository name on top of bars if height > 40,000
        repo_names = [
            f'{repo_id}' if val > 40000 else ''
            for val in data_pivot[repo_id]
        ]


        # Update the bottom_data for the next stack
        bottom_data += data_pivot[repo_id]

    # Update layout for better readability
    fig.update_layout(
        barmode='stack',
        title='DOIs Provided per Repository for Each Funder',
        xaxis_title='Funders',
        yaxis_title='Number of DOIs Provided',
        font=dict(size=18),
        hovermode='closest',
        xaxis_tickangle=-45,  # Rotate x-axis labels for readability
        width=2000,  # Set figure width
        height=1000,   # Set figure height
        plot_bgcolor='white',  # Set plot background to white
        paper_bgcolor='white',  # Set entire figure background to white
    )
        # Update x-axis and y-axis to set axis lines and ticks to light grey
    fig.update_xaxes(
        showline=True,  # Show x-axis line
        linecolor='lightgrey',  # Set x-axis line color to light grey
        showgrid=False  # Optionally remove grid lines for x-axis
    )

    fig.update_yaxes(
        showline=True,  # Show y-axis line
        linecolor='lightgrey',  # Set y-axis line color to light grey
        tickformat=',',  # Add commas to large numbers
        showgrid=False  # Optionally remove grid lines for y-axis
    )
    # Update x-axis ticks and labels
    fig.update_xaxes(
        tickvals=data_pivot.index,
        ticktext=data_pivot.index.astype(str),
        tickfont=dict(size=20)
    )

    # Update y-axis formatting to remove scientific notation and use commas
    fig.update_yaxes(tickformat=',', tickfont=dict(size=20))

    # Adjust the legend position and settings
    fig.update_layout(
        legend=dict(title='Affiliations', xanchor="auto", yanchor="top", font=dict(size=14)),
        showlegend=True  # Show filtered legend
    )

    # Show the interactive chart
    fig.show()

# Main script
if __name__ == "__main__":
    # Google Sheets ID and sheet name
    sheet_id = '1PQe813L2M-aR_5i-1gEbHtP4M4iclgdedhdrj9DuhNg'
    sheet_name = 'Funder_affiliations'
    url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

    # Load the spreadsheet
    spreadsheet = pd.read_csv(url)

    # Clean the data
    spreadsheet = clean_data(spreadsheet)

    # Pivot the data using column locations: index 0 (Funder name), columns 1 (Repository name), and values 2 (Number of DOIs provided)
    data_pivot = pd.pivot_table(spreadsheet, values=spreadsheet.columns[2],
                                index=spreadsheet.columns[0], columns=spreadsheet.columns[1], aggfunc=np.sum)

    # Fill any missing values with 0
    data_pivot = data_pivot.fillna(0)

    # Sort the pivot table by the total number of DOIs per funder in descending order
    data_pivot['Total_DOIs'] = data_pivot.sum(axis=1)  # Add a new column for total DOIs per funder
    data_pivot = data_pivot.sort_values(by='Total_DOIs', ascending=False)  # Sort by total DOIs in descending order
    data_pivot = data_pivot.drop(columns=['Total_DOIs'])  # Remove the temporary total column after sorting

    # Plot the stacked bar chart
    plot_stacked_bar_chart(data_pivot)

<ipython-input-20-477313abfeef>:116: FutureWarning:

The provided callable <function sum at 0x79bf644ab370> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.



## Relation types

In [21]:
import pandas as pd
import plotly.graph_objects as go

# The Google Sheets URL for your DOIs_OT sheet
sheet_id = '1PQe813L2M-aR_5i-1gEbHtP4M4iclgdedhdrj9DuhNg'
sheet_name = 'RelatedIdentifiers_Relationship'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

# Load the spreadsheet
spreadsheet = pd.read_csv(url)

# Extract the relevant columns by index location (0 = years, 1 = cumulative sum)
years = spreadsheet.iloc[:, 0]  # Column 0: "Years"
cumulative_sum = spreadsheet.iloc[:, 1].replace({',': ''}, regex=True).astype(int)  # Column 1: "Cumulative Sum of DOIs registered"

# Handle potential duplicates by grouping and summing (still using the location index for columns)
spreadsheet = spreadsheet.groupby(spreadsheet.iloc[:, 0]).agg({spreadsheet.columns[1]: 'sum'}).reset_index()

# Sort the data by cumulative sum in descending order
sorted_spreadsheet = spreadsheet.sort_values(by=spreadsheet.columns[1], ascending=False)

# Create variables for the sorted values
years_sorted = sorted_spreadsheet.iloc[:, 0]  # Sorted years
cumulative_sum_sorted = sorted_spreadsheet.iloc[:, 1]  # Sorted cumulative sum

# Create a vertical bar chart using Plotly
fig = go.Figure()

# Add bars to the chart
fig.add_trace(go.Bar(
    x=years_sorted,
    y=cumulative_sum_sorted,
    marker=dict(color='#243B54'),
    text=[f'{int(val):,}' for val in cumulative_sum_sorted],  # Format numbers with commas
    textposition='outside'  # Automatically position the labels
))

# Update layout for readability
fig.update_layout(
    title='Number of DOIs with corresponding Relation Type',
    xaxis_title='Relation Types',
    yaxis_title='Number of DOIs',
    font=dict(size=18),
    width=2000,  # Adjust the figure width
    height=1000,  # Adjust the figure height
    xaxis_tickangle=-45,  # Rotate x-axis labels for readability
    hovermode='closest',  # Enable unified hovermode
    plot_bgcolor='white',  # Set plot background to white
    paper_bgcolor='white',  # Set entire figure background to white
)
# Update x-axis and y-axis to set axis lines and ticks to light grey
fig.update_xaxes(
    showline=True,  # Show x-axis line
    linecolor='lightgrey',  # Set x-axis line color to light grey
    showgrid=False  # Optionally remove grid lines for x-axis
        )

fig.update_yaxes(
    showline=True,  # Show y-axis line
    linecolor='lightgrey',  # Set y-axis line color to light grey
    tickformat=',',  # Add commas to large numbers
    showgrid=False  # Optionally remove grid lines for y-axis
        )
# Update x-axis ticks and labels
fig.update_xaxes(
    tickvals=years_sorted,
    ticktext=years_sorted.astype(str),
    tickfont=dict(size=20)
)

# Update y-axis formatting to use commas
fig.update_yaxes(tickformat=',', tickfont=dict(size=20))

# Display the interactive Plotly figure
fig.show()

## Related Identifier Types

In [22]:
import pandas as pd
import plotly.graph_objects as go

# The Google Sheets URL for your DOIs_OT sheet
sheet_id = '1PQe813L2M-aR_5i-1gEbHtP4M4iclgdedhdrj9DuhNg'
sheet_name = 'RelationIdentifier_Type'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

# Load the spreadsheet
spreadsheet = pd.read_csv(url)

# Extract the relevant columns by index location (0 = years, 1 = cumulative sum)
years = spreadsheet.iloc[:, 0]  # Column 0: "Years"
cumulative_sum = spreadsheet.iloc[:, 1].replace({',': ''}, regex=True).astype(int)  # Column 1: "Cumulative Sum of DOIs registered"

# Handle potential duplicates by grouping and summing (still using the location index for columns)
spreadsheet = spreadsheet.groupby(spreadsheet.iloc[:, 0]).agg({spreadsheet.columns[1]: 'sum'}).reset_index()

# Sort the data by cumulative sum in descending order
sorted_spreadsheet = spreadsheet.sort_values(by=spreadsheet.columns[1], ascending=False)

# Create variables for the sorted values
years_sorted = sorted_spreadsheet.iloc[:, 0]  # Sorted years
cumulative_sum_sorted = sorted_spreadsheet.iloc[:, 1]  # Sorted cumulative sum

# Create a vertical bar chart using Plotly
fig = go.Figure()

# Add bars to the chart
fig.add_trace(go.Bar(
    x=years_sorted,
    y=cumulative_sum_sorted,
    marker=dict(color='#243B54'),
    text=[f'{int(val):,}' for val in cumulative_sum_sorted],  # Format numbers with commas
    textposition='outside'  # Automatically position the labels
))

# Update layout for readability
fig.update_layout(
    title='Number of DOIs with corresponding Relation Type',
    xaxis_title='Relation Types',
    yaxis_title='Number of DOIs',
    font=dict(size=18),
    width=2000,  # Adjust the figure width
    height=1000,  # Adjust the figure height
    xaxis_tickangle=-45,  # Rotate x-axis labels for readability
    hovermode='closest',  # Enable unified hovermode
    plot_bgcolor='white',  # Set plot background to white
    paper_bgcolor='white',  # Set entire figure background to white
)
# Update x-axis and y-axis to set axis lines and ticks to light grey
fig.update_xaxes(
    showline=True,  # Show x-axis line
    linecolor='lightgrey',  # Set x-axis line color to light grey
    showgrid=False  # Optionally remove grid lines for x-axis
        )

fig.update_yaxes(
    showline=True,  # Show y-axis line
    linecolor='lightgrey',  # Set y-axis line color to light grey
    tickformat=',',  # Add commas to large numbers
    showgrid=False  # Optionally remove grid lines for y-axis
        )
# Update x-axis ticks and labels
fig.update_xaxes(
    tickvals=years_sorted,
    ticktext=years_sorted.astype(str),
    tickfont=dict(size=20)
)

# Update y-axis formatting to use commas
fig.update_yaxes(tickformat=',', tickfont=dict(size=20))

# Display the interactive Plotly figure
fig.show()

## Licenses overtime

In [23]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Function to clean and format the data
def clean_data(df):
    """Cleans and formats the input DataFrame for plotting."""
    # Ensure the "Number of DOIs provided per repository" column (index 2) is numeric, remove commas if necessary
    df.iloc[:, 2] = pd.to_numeric(df.iloc[:, 2].replace({',': ''}, regex=True), errors='coerce')

    # Fill any missing values with 0
    return df.fillna(0)

# Function to plot a stacked bar chart
def plot_stacked_bar_chart(data_pivot):
    """Plots a stacked bar chart for DOI data per repository over funders."""
    # Define the color palette
    colors = ['#243b54', '#C0CED6','#46BCAB', '#0693e3', '#00B1E2', '#F07C73']

    # Create the Plotly figure
    fig = go.Figure()

    # Initialize bottom_data for stacking the bars
    bottom_data = np.zeros(len(data_pivot))

    # Loop through each repository and add bars
    for idx, repo_id in enumerate(data_pivot.columns):
        total_contribution = data_pivot[repo_id].sum()

        # Only show repositories in the legend if they contributed more than 40,000 DOIs
        show_legend = bool(total_contribution > 460000)

        # Add bars to the chart
        fig.add_trace(go.Bar(
            x=data_pivot.index,
            y=data_pivot[repo_id],
            name=repo_id if show_legend else '',  # Only show in legend if contribution > 40,000
            marker=dict(color=colors[(idx + 2) % len(colors)]),
            text=[f'{int(val):,}' if val > 400000 else '' for val in data_pivot[repo_id]],  # Display value if > 40,000
            textposition='auto',  # Automatically place the text
            textangle=45,  # Rotate the text by 45 degrees
            hovertemplate=(
                f'<b>License Type:</b> {repo_id}<br>' +
                '<b>Year:</b> %{x}<br>' +
                '<b>Number of DOIs:</b> %{y:,}<extra></extra>'
            ),
            showlegend=show_legend  # Show in legend only if contribution > 40,000
        ))

        # Add repository name on top of bars if height > 40,000
        repo_names = [
            f'{repo_id}' if val > 400000 else ''
            for val in data_pivot[repo_id]
        ]


        # Update the bottom_data for the next stack
        bottom_data += data_pivot[repo_id]

    # Update layout for better readability
    fig.update_layout(
        barmode='stack',
        #title='DOIs Provided per Repository for Each Funder',
        xaxis_title='Years',
        yaxis_title='Number of DOIs',
        font=dict(size=18),
        hovermode='closest',
        xaxis_tickangle=-45,  # Rotate x-axis labels for readability
        width=2000,  # Set figure width
        height=1000,   # Set figure height
        plot_bgcolor='white',  # Set plot background to white
        paper_bgcolor='white',  # Set entire figure background to white
    )
        # Update x-axis and y-axis to set axis lines and ticks to light grey
    fig.update_xaxes(
        showline=True,  # Show x-axis line
        linecolor='lightgrey',  # Set x-axis line color to light grey
        showgrid=False  # Optionally remove grid lines for x-axis
    )

    fig.update_yaxes(
        showline=True,  # Show y-axis line
        linecolor='lightgrey',  # Set y-axis line color to light grey
        tickformat=',',  # Add commas to large numbers
        showgrid=False  # Optionally remove grid lines for y-axis
    )
    # Update x-axis ticks and labels
    fig.update_xaxes(
        tickvals=data_pivot.index,
        ticktext=data_pivot.index.astype(str),
        tickfont=dict(size=20)
    )

    # Update y-axis formatting to remove scientific notation and use commas
    fig.update_yaxes(tickformat=',', tickfont=dict(size=20))

    # Adjust the legend position and settings
    fig.update_layout(
        legend=dict(title='License Type', x=0, y=1, xanchor="left", yanchor="top", font=dict(size=14)),
        showlegend=True  # Show filtered legend
    )

    # Show the interactive chart
    fig.show()

# Main script
if __name__ == "__main__":
    # Google Sheets ID and sheet name
    sheet_id = '1PQe813L2M-aR_5i-1gEbHtP4M4iclgdedhdrj9DuhNg'
    sheet_name = 'License_OT'
    url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

    # Load the spreadsheet
    spreadsheet = pd.read_csv(url)

    # Clean the data
    spreadsheet = clean_data(spreadsheet)

    # Pivot the data using column locations: index 0 (Funder name), columns 1 (Repository name), and values 2 (Number of DOIs provided)
    data_pivot = pd.pivot_table(spreadsheet, values=spreadsheet.columns[2],
                                index=spreadsheet.columns[0], columns=spreadsheet.columns[1], aggfunc=np.sum)

    # Fill any missing values with 0
    data_pivot = data_pivot.fillna(0)

    # Sort the pivot table by the total number of DOIs per funder in descending order
    data_pivot['Total_DOIs'] = data_pivot.sum(axis=1)  # Add a new column for total DOIs per funder
    data_pivot = data_pivot.sort_values(by='Total_DOIs', ascending=False)  # Sort by total DOIs in descending order
    data_pivot = data_pivot.drop(columns=['Total_DOIs'])  # Remove the temporary total column after sorting

    # Plot the stacked bar chart
    plot_stacked_bar_chart(data_pivot)

<ipython-input-23-c1ad8ae7ae71>:120: FutureWarning:

The provided callable <function sum at 0x79bf644ab370> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.

